In [1]:
from binascii import hexlify, unhexlify
cal_msg = b'\xFF\x06\x00\x0A\x00\x06\x3C\x14'
cal_resp = b'\xff\x06\x00\n\x00\x06<\x14'
ver_msg = b'\xfe\x03\x00\n\x00\x01\xb0\x07'
ver_resp = b'\xfe\x03\x02\x00=m\x81'

In [2]:
print(hexlify(cal_msg[:]))
print(hexlify(cal_resp[:]))
print(hexlify(ver_msg[:]))
print(hexlify(ver_resp[:]))

b'ff06000a00063c14'
b'ff06000a00063c14'
b'fe03000a0001b007'
b'fe0302003d6d81'


In [8]:
class modbus_message:
    def __init__(self, raw_data) -> None:
        self.raw_data = hexlify(raw_data)
        self.station_address = self.raw_data[0:2]
        self.function_code = self.raw_data[2:4]  
        self.payload = self.raw_data[4:-4]
        self.crc = self.raw_data[-4:]
    def __str__(self) -> str:
        return f"Station Address:{self.station_address} Function Code:{self.function_code} CRC:{self.crc}"

class modbus_read_message(modbus_message):
    def __init__(self, raw_data) -> None:
        super().__init__(raw_data)
        self.register_start_address = self.payload[0:4]
        self.n_registers = self.payload[4:8]
    def __str__(self) -> str:
        return super().__str__() + f" Register Start Address:{self.register_start_address} N Registers:{self.n_registers}"

class modbus_write_single_message(modbus_message):
    def __init__(self, raw_data) -> None:
        super().__init__(raw_data)
        if len(self.payload) != 8:
            raise ValueError(f"Payload must be 8 bytes long, is {len(self.payload)}")
        self.register_address = self.payload[0:4]
        self.register_value = self.payload[4:8]
    def __str__(self) -> str:
        return super().__str__() + f" Register Address:{self.register_address} Register Value:{self.register_value}"

class modbus_write_multiple_message(modbus_message):
    def __init__(self, raw_data) -> None:
        super().__init__(raw_data)
        if len(self.payload) < 6:
            raise ValueError("Payload must be at least 6 bytes long")
        self.register_start_address = self.payload[0:4]
        self.n_registers = self.payload[4:8]
        self.byte_count = self.payload[8:10]
        self.data = self.payload[10:]
    def __str__(self) -> str:
        return super().__str__() + f" Register Start Address:{self.register_start_address} N Registers:{self.n_registers} Byte Count:{self.byte_count} Data:{self.data}"

In [17]:
class modbus_response:
    def __init__(self, raw_data) -> None:
        self.raw_data = hexlify(raw_data)
        self.station_address = self.raw_data[0:2]
        self.function_code = self.raw_data[2:4]  
        self.payload = self.raw_data[4:-4]
        self.crc = self.raw_data[-4:]
    def __str__(self):
        return f'Station Address:{self.station_address} Function Code:{self.function_code} CRC:{self.crc}'

In [18]:
class modbus_read_response(modbus_response):
    def __init__(self, raw_data) -> None:
        super().__init__(raw_data)
        self.byte_count = self.payload[0:2]
        self.data = self.payload[2:]
    def __str__(self):
        return super().__str__() + f' Byte Count:{self.byte_count} Data:{self.data}'

class modbus_write_single_response(modbus_response):
    def __init__(self, raw_data) -> None:
        super().__init__(raw_data)
        self.address = self.payload[0:4]
        self.value = self.payload[4:]
    def __str__(self):
        return super().__str__() + f' Address:{self.address} Value:{self.value}'
    
class modbus_write_multiple_response(modbus_response):
    def __init__(self, raw_data) -> None:
        super().__init__(raw_data)
        self.address = self.payload[0:4]
        self.n_registers = self.payload[4:8]
    def __str__(self):
        return super().__str__() + f' Address:{self.address} N Registers:{self.n_registers}'

In [19]:
cal_msg_o = modbus_write_single_message(cal_msg)
cal_resp_o = modbus_write_single_response(cal_resp)
ver_msg_o = modbus_read_message(ver_msg)
ver_resp_o = modbus_read_response(ver_resp)

In [20]:
print(cal_msg_o)
print(cal_resp_o)
print(ver_msg_o)
print(ver_resp_o)

Station Address:b'ff' Function Code:b'06' CRC:b'3c14' Register Address:b'000a' Register Value:b'0006'
Station Address:b'ff' Function Code:b'06' CRC:b'3c14' Address:b'000a' Value:b'0006'
Station Address:b'fe' Function Code:b'03' CRC:b'b007' Register Start Address:b'000a' N Registers:b'0001'
Station Address:b'fe' Function Code:b'03' CRC:b'6d81' Byte Count:b'02' Data:b'003d'


In [28]:
# show base 10 representation of ver_resp_o.data
print(int(ver_resp_o.data, 16))

61
